In [3]:
# test reading in 10x data with AnnData
import pandas as pd
import numpy as np
import scanpy as sc
from glob import glob
import os
import pickle

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_palette(sns.color_palette("Spectral"))
sns.set_style("whitegrid")

from matplotlib.ticker import StrMethodFormatter
# ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.2f}"))

In [2]:
data_dir = "data"

In [ ]:
for i,j,k in os.walk(data_dir):
    print(i,j,k)
    print('\n')

data\GSE154775\GSM4679492_HS1
data\GSE154775\GSM4679493_HS2
data\GSE154775\GSM4679494_181009_HS_lesional_epi
data\GSE154775\GSM4679495_181009_HS_lesional_dermis
data\GSE154775\GSM4679496_HSEpi190103
data\GSE154775\GSM4679497_HSDerm190103
data\GSE154775\GSM4679498_HSEpi190115
data\GSE154775\GSM4679499_HSDerm190115
data\GSE154775\GSM4679500_HS_07_2018-06-29
data\GSE155850\GSM4712969_Healthy_1
data\GSE155850\GSM4712970_Healthy_2
data\GSE155850\GSM4712971_HS_1
data\GSE155850\GSM4712972_HS_2
data\GSE155850\GSM7754887_Healthy1KC
data\GSE155850\GSM7754888_Healthy2KC
data\GSE155850\GSM7754889_HS1KC
data\GSE155850\GSM7754890_HS2KC
data\GSE175990\GSM5352392_HS_1
data\GSE175990\GSM5352393_HS_2
data\GSE175990\GSM5352394_HS_3
data\GSE175990\GSM5352395_HC
data\GSE220116\GSM6840117_HIS_DIS_01M_LS_Der
data\GSE220116\GSM6840118_HIS_DIS_01M_LS_Epi
data\GSE220116\GSM6840119_HIS_DIS_02M_LS_Der
data\GSE220116\GSM6840120_HIS_DIS_02M_LS_Epi
data\GSE220116\GSM6840121_HIS_DIS_A_LS_Total
data\GSE220116\GSM68401

In [16]:
# Function is beyond broken
# scanpy.read_10x_mtx("data_extract/GSE154775/GSM4679492_HS1")

# make dictionary of gsm code and path
gsm_dict = {}
for root, folders, _ in os.walk(data_dir):
    for i in folders:
        if i.startswith("GSM"):
            gsm_dict[i.split("_")[0]] = os.path.join(root,i)

# returns full path from GSM number
def gsm_path(gsm):
    return gsm_dict["GSM" + str(gsm)]

# returns anndata object of 10x data
def read_dir(gsm_code):
    data_path = gsm_path(gsm_code)
    matrix_path = os.path.join(data_path, 'matrix.mtx')
    barcodes_path = os.path.join(data_path, 'barcodes.tsv')
    features_path = os.path.join(data_path, 'features.tsv')
    
    adata = sc.read_mtx(matrix_path)
    adata_bc = pd.read_csv(barcodes_path, header=None, delimiter = "\t")
    adata_features = pd.read_csv(features_path, header=None, delimiter = '\t')
    print(adata_features)
    
    adata = adata.T
    adata.obs['cell_id'] = adata_bc
    adata.var['gene_name'] = adata_features[1].values
    adata.var.index = adata.var['gene_name']
    
    return adata



In [23]:
df = read_dir(4679493)

                     0             1
0      ENSG00000243485    MIR1302-10
1      ENSG00000237613       FAM138A
2      ENSG00000186092         OR4F5
3      ENSG00000238009  RP11-34P13.7
4      ENSG00000239945  RP11-34P13.8
...                ...           ...
32733  ENSG00000215635    AC145205.1
32734  ENSG00000268590         BAGE5
32735  ENSG00000251180    CU459201.1
32736  ENSG00000215616    AC002321.2
32737  ENSG00000215611    AC002321.1

[32738 rows x 2 columns]


In [33]:
df.X[1:10, 1:10]

SyntaxError: invalid syntax (1458202448.py, line 1)

In [104]:
df.var["mt"] = df.var_names.str.startswith("MT-")
df.var["ribo"] = df.var_names.str.startswith(("RPS", "RPL"))

In [111]:
type(df.var)

pandas.core.frame.DataFrame

In [44]:
pd.read_csv("data_extract/GSE154775/GSM4679492_HS1/features.tsv", delimiter = '\t', header=None)

,0,1
0,ENSG00000243485,MIR1302-10
1,ENSG00000237613,FAM138A
2,ENSG00000186092,OR4F5
3,ENSG00000238009,RP11-34P13.7
4,ENSG00000239945,RP11-34P13.8
...,...,...
32733,ENSG00000215635,AC145205.1
32734,ENSG00000268590,BAGE5
32735,ENSG00000251180,CU459201.1
32736,ENSG00000215616,AC002321.2


# Preprocessing Steps
1. load data
2. QC, for each sample, filter for cells with:
    a. 300 < Unique Genes < 6000
    b. total RNA count < 40000
    c. Michtocondrial DNA < 20%
    d. Perform doublet detection and drop doublet cells
    e. merge by concatenate
    f. normalize

